In [1]:
import numpy as np

from utils import GamblerProblemModel

In [2]:
p = GamblerProblemModel(goal=10, success_probability=0.4)
p.n_states, p.n_nonterminal_states

(11, 9)

In [3]:
policy = p.propose_policy_estimate()
policy

array([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [4]:
ptpm = p.define_ptpm(policy)
ptpm

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.6, 0. , 0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.6, 0. , 0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.6, 0. , 0.4, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.6, 0. , 0.4, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.6, 0. , 0.4, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.4, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.4, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.4, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0.4],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

In [5]:
immediate_rewards = p.immediate_rewards
immediate_rewards

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [6]:
DISCOUNT = 1

In [7]:
values_of_states = np.zeros(p.n_states)
for i in range(10):
    new_values_of_states = immediate_rewards + DISCOUNT * np.matmul(ptpm, values_of_states)
    new_values_of_states = np.round(new_values_of_states, 4)
    err = (np.abs(values_of_states - new_values_of_states)).sum()
    print(new_values_of_states, err)
    values_of_states = new_values_of_states

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] 1.0
[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.4 1. ] 0.4
[0.   0.   0.   0.   0.   0.   0.   0.   0.16 0.4  1.  ] 0.16
[0.    0.    0.    0.    0.    0.    0.    0.064 0.16  0.496 1.   ] 0.15999999999999998
[0.     0.     0.     0.     0.     0.     0.0256 0.064  0.2368 0.496
 1.    ] 0.1024
[0.     0.     0.     0.     0.     0.0102 0.0256 0.1101 0.2368 0.5421
 1.    ] 0.10240000000000003
[0.     0.     0.     0.     0.0041 0.0102 0.0502 0.1101 0.2829 0.5421
 1.    ] 0.07479999999999998
[0.     0.     0.     0.0016 0.0041 0.0225 0.0502 0.1433 0.2829 0.5697
 1.    ] 0.07469999999999996
[0.000e+00 0.000e+00 6.000e-04 1.600e-03 1.000e-02 2.250e-02 7.080e-02
 1.433e-01 3.139e-01 5.697e-01 1.000e+00] 0.05810000000000003
[0.000e+00 2.000e-04 6.000e-04 4.400e-03 1.000e-02 3.430e-02 7.080e-02
 1.680e-01 3.139e-01 5.883e-01 1.000e+00] 0.05810000000000006


In [9]:
a = np.arange(12).reshape(3, 4)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [10]:
b = np.arange(4)

In [11]:
np.matmul(a, b)

array([14, 38, 62])

In [12]:
np.matvec(a, b)

array([14, 38, 62])

In [15]:
aa = np.stack([a, a])
print(aa.shape)
aa

(2, 3, 4)


array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]])

In [17]:
r = np.matvec(aa, b)
print(r.shape)
r

(2, 3)


array([[14, 38, 62],
       [14, 38, 62]])

In [67]:
def define_transition_probability_matrices(n_actions, n_states, p_success):
    p_failure = 1 - p_success
    transition_probs = np.zeros((n_actions, n_states, n_states))
    for action_idx in range(n_actions):
        action_value = action_idx + 1
        for state in range(n_states):
            if state - action_value < 0 or state + action_value >= n_states:
                continue
            transition_probs[action_idx, state, state - action_value] = p_failure
            transition_probs[action_idx, state, state + action_value] = p_success
    return transition_probs

In [68]:
tps100 = define_transition_probability_matrices(50, 101, 0.4)
tps100[49].sum(axis=1)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [72]:
tps = define_transition_probability_matrices(5, 11, 0.4)


first = np.matvec(tps, immediate_rewards)
np.argmax(first, axis=0)

array([0, 0, 0, 0, 0, 4, 3, 2, 1, 0, 0])

In [73]:
np.max(first, axis=0)

array([0. , 0. , 0. , 0. , 0. , 0.4, 0.4, 0.4, 0.4, 0.4, 0. ])

In [74]:
first

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.4, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.4, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.4, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.4, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.4, 0. , 0. , 0. , 0. , 0. ]])

In [76]:
v = np.zeros(11)
for i in range(10):
    comp = np.matvec(tps, v+immediate_rewards)
    print(np.argmax(comp, axis=0))
    v = np.max(comp, axis=0)
    print(v)
    print()

[0 0 0 0 0 4 3 2 1 0 0]
[0.  0.  0.  0.  0.  0.4 0.4 0.4 0.4 0.4 0. ]

[0 0 0 1 0 4 0 0 1 0 0]
[0.   0.   0.   0.16 0.16 0.4  0.4  0.4  0.64 0.64 0.  ]

[0 0 0 1 0 4 0 2 1 0 0]
[0.    0.    0.064 0.16  0.256 0.4   0.4   0.496 0.64  0.784 0.   ]

[0 0 1 1 0 4 0 1 1 0 0]
[0.     0.0256 0.1024 0.16   0.256  0.4    0.4384 0.5536 0.64   0.784
 0.    ]

[0 0 1 1 0 4 0 1 1 0 0]
[0.      0.04096 0.1024  0.17536 0.256   0.4     0.46144 0.5536  0.66304
 0.784   0.     ]

[0 0 1 1 0 4 0 1 1 0 0]
[0.       0.04096  0.1024   0.184576 0.265216 0.4      0.46144  0.5536
 0.676864 0.797824 0.      ]

[0 0 1 1 3 4 0 1 1 0 0]
[0.        0.04096   0.1060864 0.184576  0.2707456 0.4       0.46144
 0.5591296 0.676864  0.8061184 0.       ]

[0 0 1 1 3 4 0 1 1 0 0]
[0.         0.04243456 0.10829824 0.184576   0.2707456  0.4
 0.46365184 0.56244736 0.676864   0.8061184  0.        ]

[0 0 1 1 3 4 0 1 1 0 0]
[0.         0.0433193  0.10829824 0.18546074 0.2707456  0.4
 0.46497894 0.56244736 0.6781911  0.8061184  0.